In [1]:
######################

#Download "data/mastr/powerProduction_alldata.csv" from the our google drive (too large for git)

#####################

import xml.etree.ElementTree as Xet
import pandas as pd
import numpy as np

#please download this file from our google drive

df_org = pd.read_csv("data/mastr/powerProduction_alldata.csv")
df_org

/var/folders/6w/mz49x3ds0b18vg8p9jx7wg0r0000gn/T/ipykernel_3156/1326151739.py:13: DtypeWarning: Columns (8,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df_org = pd.read_csv("data/mastr/powerProduction_alldata.csv")


,Unnamed: 0.1,Unnamed: 0,energy,land,bundesland,landkreis,gemeinde,gemeinde_id,landkreis_id,laengengrad,...,inbetriebnahmedatum,datumEndgueltigeStilllegung,bruttoleistung,nettonennleistung,nameEinheit,betriebsstatus,energietraeger,wind_kraftwerk,wind_nordsee,wind_ostsee
0,0,0,solar,84,1408.0,Region Hannover,Seelze,03241015id,3241,NaN,...,2017-11-17,NaN,6.00,5.00,Photovoltaik Dedensen,35,2495,NaN,NaN,NaN
1,1,1,solar,84,1409.0,Essen,Essen,05113000id,5113,NaN,...,2007-06-01,NaN,3.24,3.24,Hausdach,35,2495,NaN,NaN,NaN
2,2,2,solar,84,1408.0,Osnabrück,Merzen,03459026id,3459,NaN,...,2014-07-17,NaN,8.30,8.30,Fronius Symo,35,2495,NaN,NaN,NaN
3,3,3,solar,84,1406.0,Hamburg,Hamburg,02000000id,2000,NaN,...,2008-08-27,NaN,2.92,2.92,Hausdach,35,2495,NaN,NaN,NaN
4,4,4,solar,84,1403.0,Bayreuth,Hollfeld,09472154id,9472,NaN,...,2013-06-18,NaN,5.50,5.00,"Anlage-1-PV-5,5kwp",35,2495,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2515865,1,1,kernkraft,84,1411.0,Steinburg,Brokdorf,01061018id,1061,9.344720,...,1986-10-14,2021-12-31,1480000.00,1410000.00,Brokdorf,38,2494,NaN,NaN,NaN
2515866,2,2,kernkraft,84,1403.0,Landshut,Essenbach,09274128id,9274,12.293150,...,1988-01-22,NaN,1485000.00,1410000.00,Isar 2,35,2494,NaN,NaN,NaN
2515867,3,3,kernkraft,84,1403.0,Günzburg,Gundremmingen,09774136id,9774,10.404743,...,1984-11-02,2021-12-31,1344000.00,1288000.00,Gundremmingen C,38,2494,NaN,NaN,NaN
2515868,4,4,kernkraft,84,1408.0,Emsland,Lingen (Ems),03454032id,3454,7.324140,...,1988-06-20,NaN,1406000.00,1336000.00,Emsland A,35,2494,NaN,NaN,NaN


In [2]:
df = df_org.copy();

#only producers currently producing
df = df[df.betriebsstatus == 35]

#only registered in germany
df = df[df.land == 84]

#solve the problem with the kreisstädte double-naming
#exclude borken (see below)
kreisstaedte = ["Ansbach","Aschaffenburg","Augsburg","Bamberg","Bayreuth","Coburg","Fürth","Heilbronn","Hof","Kaiserslautern","Karlsruhe","Kassel","Landshut","Leipzig","München","Oldenburg","Osnabrück","Passau","Regensburg","Rosenheim","Rostock","Schweinfurt","Würzburg"]

def check_city(row):
    kreisstadt = row["landkreis"]
    if kreisstadt != row["gemeinde"]:
        row.landkreis = "Kreis " + kreisstadt
    else:
        row.landkreis = kreisstadt
    return row

df.update(df.loc[df['landkreis'].isin(kreisstaedte)].apply(
    lambda row: check_city(row),axis=1))


#correct mistakes in datawrapper map (germany districts 2021)
#df["landkreis"] = df['landkreis'].str.replace('Eisenach','Eifelkreis Bitburg-Prüm')
df["landkreis"] = df['landkreis'].str.replace('Borken','Kreis Borken') 

#correct mistakes in dataset
df["landkreis"] = df['landkreis'].str.replace('Osterode am Harz','Göttingen')
df.loc[df["landkreis"] == "Göttingen", "landkreis_id"] = 3159
#TODO: göttingen is not really working

#offshore windparks
df.loc[(df["wind_nordsee"] > 0) & (df["energy"] == "wind"), "landkreis"] = "offshoreNordsee"
df.loc[(df["wind_nordsee"] > 0) & (df["energy"] == "wind"), "landkreis_id"] = 000
df.loc[(df["wind_ostsee"] > 0) & (df["energy"] == "wind"), "landkreis"] = "offshoreOstsee"
df.loc[(df["wind_ostsee"] > 0) & (df["energy"] == "wind"), "landkreis_id"] = 000

other_offshore = ["11WD8BALT3W----6","BALTICERZ"]
df.loc[df["wind_kraftwerk"].isin(other_offshore) & (df["energy"] == "wind"), "landkreis_id"] = 000
df.loc[df["wind_kraftwerk"].isin(other_offshore) & (df["energy"] == "wind"), "landkreis"] = "offshoreOstsee"

#create list of other offshore unit names
x = 10
list = (["HS A"] * x) + ["HS B"] * x + (["HS C"] * x) + (["HS D"] * x) + (["HS E"] * x) + ["HS F"] * x + ["HS G"] * x
list += ["HS H"] * x + ["HS I"] * x + ["HS J"] * x + ["HS K"] * x + ["HS L"] * x + ["HS M"] * x + ["HS N"] * x + ["HS O"] * x
list += ["HS P"] * x + ["HS Q"] * x + ["HS R"] * x 

other_offshore_einheit = [item + str(i%10) for (i,item) in enumerate(list)]
other_offshore_einheit += ["AL 0" + str(i) for i in range(10)] + ["AL " + str(i) for i in range(10,60)]
other_offshore_einheit += ["HS I3a","HS H3a"]

df.loc[df["nameEinheit"].isin(other_offshore_einheit) & (df["energy"] == "wind"), "landkreis_id"] = 000
df.loc[df["nameEinheit"].isin(other_offshore_einheit) & (df["energy"] == "wind"), "landkreis"] = "offshoreNordsee"

#remove weird small things without landkreis
weird_things = ["Jan Wilkens Bioenergie","Pflanzenöl BHKW"]
df = df[~df["nameEinheit"].isin(weird_things)]

df["landkreis_id"] = df["landkreis_id"].astype(int)


In [3]:
#store all data with geo coordinates in file, so I can load in qgis
geo = df.copy()
geo = geo[geo["breitengrad"].notna()]
geo.to_csv("data/mastr/powerproduction_germany_geo.csv")

In [4]:
#this file was created with qgis
geo_to_remove = pd.read_csv("data/mastr/powerProductionOutsideGermany.csv")

#keep offshore cluster einheiten from remove list
geo_to_remove = geo_to_remove[(geo_to_remove.landkreis != "offshoreNordsee") & (geo_to_remove.landkreis != "offshoreOstsee")]

geo_to_remove

#drop data points outside of germany
cond = (df['laengengrad'].isin(geo_to_remove['laengengrad'])) & (df['breitengrad'].isin(geo_to_remove['breitengrad']))
df_germany = df.drop(df[cond].index)
df_germany.to_csv("data/mastr/powerProductionGermany.csv")


In [5]:
# bruttoleistung of different energy producers per location
# Create pivot table

print(df_germany["energy"].unique())


def createEnergyPerLandkreis(isBrutto):

    if isBrutto:
        value = 'bruttoleistung'
    else:
        value = "nettonennleistung"
    
    energyTotal = pd.pivot_table(
        df_germany,
        index='energy',
        columns=['landkreis',"landkreis_id"],
        values=value,
        aggfunc=np.sum,
        margins=False
    )

    energyTotal = energyTotal.rename_axis(None).transpose().reset_index(level=0).fillna(0)

    #sum up different energy types
    energyTotal["renewable_total"] = energyTotal[['solar','wasser','wind','biomasse',"geoSolarthermieGrubenKlaerschlammDruckentspannung"]].agg('sum', axis=1)
    energyTotal["not_renewable_total"] = energyTotal[['verbrennung','kernkraft']].agg('sum', axis=1)

    #percentage of renewables in landkreis
    energyTotal["percRenewableinRegion"] = energyTotal["renewable_total"] / (energyTotal["not_renewable_total"]+energyTotal["renewable_total"]) * 100
    
    energyTotal = energyTotal.reset_index(level = 0)
    energyTotal["landkreis_id"] = energyTotal["landkreis_id"].astype(int)
    
    return energyTotal


brutto = createEnergyPerLandkreis(True)
brutto.to_csv("data/mastr/powerPerLandkreis_brutto.csv")
brutto

netto = createEnergyPerLandkreis(True).reset_index(level=0)
netto.to_csv("data/mastr/powerPerLandkreis_netto.csv")
netto

#print(netto.landkreis.unique())
#print(netto[netto.landkreis == "offshoreNordsee"])



['solar' 'verbrennung' 'wasser' 'wind' 'biomasse'
 'geoSolarthermieGrubenKlaerschlammDruckentspannung' 'kernkraft']


,index,landkreis_id,landkreis,biomasse,geoSolarthermieGrubenKlaerschlammDruckentspannung,kernkraft,solar,verbrennung,wasser,wind,renewable_total,not_renewable_total,percRenewableinRegion
0,0,7131,Ahrweiler,2909.00,0.0,0.0,58852.747,6348.620,60.0,15690.0,77511.747,6348.620,92.429535
1,1,9771,Aichach-Friedberg,48984.60,0.0,0.0,261294.576,2762.600,28680.6,21612.0,360571.776,2762.600,99.239654
2,2,8425,Alb-Donau-Kreis,58418.65,110.0,0.0,355201.385,24586.285,8495.5,32691.0,454916.535,24586.285,94.872546
3,3,16077,Altenburger Land,17115.00,0.0,0.0,153066.829,8865.754,261.0,116525.0,286967.829,8865.754,97.003128
4,4,7132,Altenkirchen (Westerwald),1292.00,140.0,0.0,66616.515,1905.830,1633.0,19300.0,88981.515,1905.830,97.903085
...,...,...,...,...,...,...,...,...,...,...,...,...,...
398,398,8417,Zollernalbkreis,9518.00,0.0,0.0,171624.423,27281.720,1189.6,1815.4,184147.423,27281.720,87.096519
399,399,7320,Zweibrücken,2695.00,0.0,0.0,17277.108,2691.070,215.0,0.0,20187.108,2691.070,88.237394
400,400,14524,Zwickau,20443.30,80.0,0.0,155749.905,25909.650,6209.0,90975.2,273457.405,25909.650,91.345190
401,401,0,offshoreNordsee,0.00,0.0,0.0,0.000,0.000,0.0,6690668.0,6690668.000,0.000,100.000000


In [6]:
#area size dataset
area = pd.read_csv("../general_data/area.csv",delimiter=";")
area

#drop unimportant columns and prepare dataset
area.drop(area.columns.difference(['Schlüssel-nummer','Kreis/Landkreis',"Fläche\ninkm2","Bevölkerunginsgesamt"]), 1, inplace=True)
area = area.rename(columns={"Fläche\ninkm2": 'area'})
area['area'] = area['area'].str.replace(',','.')
area["area"] = pd.to_numeric(area["area"])
print(area)

#merge datasets, keep offshore
df2 = pd.merge(area,netto,how="outer",left_on="Schlüssel-nummer",right_on="landkreis_id")

#calculate renewable energy per km2
df2["renewable_pro_qkm"] = df2["renewable_total"] / df2["area"]
df2["solar_pro_qkm"] = df2["solar"] / df2["area"]
df2["wind_pro_qkm"] = df2["wind"] / df2["area"]

df2.to_csv("data/mastr/powerPerLandkreis_sqm_netto.csv")


     Schlüssel-nummer        Kreis/Landkreis     area  Bevölkerunginsgesamt
0                1001        Flensburg,Stadt    56.73                 89934
1                1002  Kiel,Landeshauptstadt   118.65                246601
2                1003      Lübeck,Hansestadt   214.19                215846
3                1004       Neumünster,Stadt    71.66                 79905
4                1051           Dithmarschen  1428.17                133251
..                ...                    ...      ...                   ...
396             16073    Saalfeld-Rudolstadt  1008.79                102139
397             16074   Saale-Holzland-Kreis   815.24                 82816
398             16075       Saale-Orla-Kreis  1151.30                 79632
399             16076                  Greiz   845.98                 96668
400             16077        AltenburgerLand   569.39                 88356

[401 rows x 4 columns]


/var/folders/6w/mz49x3ds0b18vg8p9jx7wg0r0000gn/T/ipykernel_3156/3347863731.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  area.drop(area.columns.difference(['Schlüssel-nummer','Kreis/Landkreis',"Fläche\ninkm2","Bevölkerunginsgesamt"]), 1, inplace=True)


In [7]:
#just some random testing and analyzing
df2 = pd.read_csv("data/mastr/powerPerLandkreis_sqm_netto.csv")

#calculate renewable energy per km2
df2["renewable_per_capita"] = df2["renewable_total"] / df2["Bevölkerunginsgesamt"]


#df2.to_csv("data/mastr/powerPerLandkreis_per_person.csv")

#df2[df2.landkreis == "Barnim"]
df2.loc[df2["wind_pro_qkm"].idxmax()]
#df2.loc[df2["wind_pro_qkm"] == 0.0]

Unnamed: 0                                                    45
Schlüssel-nummer                                          3402.0
Kreis/Landkreis                                      Emden,Stadt
area                                                      112.34
Bevölkerunginsgesamt                                     49874.0
index                                                         78
landkreis_id                                                3402
landkreis                                                  Emden
biomasse                                                 22075.0
geoSolarthermieGrubenKlaerschlammDruckentspannung            0.0
kernkraft                                                    0.0
solar                                                  16886.578
verbrennung                                             56536.95
wasser                                                       0.0
wind                                                    186660.0
renewable_total          